## xcube Data Cube Generator Service

This notebook provides a walk-through demonstrating how to use the _data cube generator service_.

The data cube generator service demonstrated here is a Python client for the [xcube service generator API](https://stage.xcube-gen.brockmann-consult.de/api/v2/ui/).

The Python client is represented by the calss `CubeGeneratorService`. Instances of this are created by passing it a 
`CubeGeneratorRequest` and a `ServiceConfig` instance. Here are imports for the named classes:

In [9]:
from xcube.core.gen2 import CubeGeneratorRequest
from xcube.core.gen2.service import CubeGeneratorService
from xcube.core.gen2.service import ServiceConfig

We first need to provide `client_id` and `client_secret` to authorize for using the service. 
An `endpoint_url` is optional and defaults to the standard endpoint for the EuroDataCube cube generation service.

In [2]:
service_config_json = {
    "client_id": "...",
    "client_secret": "..."
}

However, it is much saver to ingest service configuration from a JSON or YAML file (the file path used must be adjusted):

In [3]:
import yaml
with open('../../../edc-service.yml') as fp:
    service_config_json = yaml.safe_load(fp)

Validate and convert `service_config_json` to a service configuration object:

In [10]:
service_config = ServiceConfig.from_dict(service_config_json)

Now we define the _cube generator request_. 
The following example is supposed to read an `S2L2A` dataset from the Sentinel Hub data store (`input_config`), 
using numerous target cube parameters(`cube_config`), and
writes the target cube to a well-known scratch bucket in AWS S3 (`output_config`).

This is how a _cube generator request_ looks as a (JSON) dictionary:

In [11]:
request_json = {
  "input_config": {
    "store_id": "@sentinelhub_eu",
    "data_id": "S2L2A"
  },
  "cube_config": {
    "variable_names": ["B04", "B05"],
    "bbox": [7, 53, 9, 55],
    "spatial_res": 0.001,
    "tile_size": [1024, 1024],
    "crs": "WGS84",
    "time_range": ["2019-08-05", "2019-08-10"],
    "time_period": "1D"
  },
  "output_config": {
    "store_id": "@eurodatacube_scratch",
    "data_id": "S2L2A_B04_B05_7_53_9_55.zarr",
    "replace": True
  }
}

Validate and convert `request_json` to a request object:

In [12]:
request = CubeGeneratorRequest.from_dict(request_json)
request

Instantiate the generator object:

In [13]:
gen = CubeGeneratorService(request, service_config, verbosity=1)

Get some information about the cube that would be generated by `gen` (this may take several seconds to complete):

In [14]:
cube_info = gen.get_cube_info()
cube_info

RuntimeError: internal error: unexpected response from API call https://stage.xcube-gen.brockmann-consult.de/api/v2/cubegens/info: Additional properties are not allowed ('result' was unexpected)

Failed validating 'additionalProperties' in schema:
    {'additionalProperties': False,
     'properties': {'cost_estimation': {'additionalProperties': True,
                                        'type': 'object'},
                    'dataset_descriptor': {'additionalProperties': False,
                                           'properties': {'attrs': {'additionalProperties': True,
                                                                    'type': 'object'},
                                                          'bbox': {'items': [{'type': 'number'},
                                                                             {'type': 'number'},
                                                                             {'type': 'number'},
                                     

Now perform the actual cube generation:

In [15]:
cube_id = gen.generate_cube()
cube_id

RuntimeError: internal error: unexpected response from API call https://stage.xcube-gen.brockmann-consult.de/api/v2/cubegens: 'status' is a required property

Failed validating 'required' in schema:
    {'additionalProperties': True,
     'properties': {'cubegen_id': {'minLength': 1, 'type': 'string'},
                    'output': {'items': {'type': 'string'},
                               'type': ['array', 'null']},
                    'progress': {'items': {'additionalProperties': False,
                                           'properties': {'sender': {'type': 'string'},
                                                          'state': {'additionalProperties': True,
                                                                    'properties': {'progress': {'minimum': 0.0,
                                                                                                'type': 'number'},
                                                                                   'total_

Let's open the generated cube:

In [ ]:
from xcube.core.dsio import open_cube

In [ ]:
ds = open_cube(f'https://s3.eu-central-1.amazonaws.com/eurodatacube-scratch/{cube_id}', s3_kwargs={"anon": True}))
ds